In [1]:
import geopandas as gpd

**Intro to QuackOSM**

In [2]:
import quackosm as qosm
qosm.convert_osm_extract_to_geodataframe("Vatican City")

,tags,geometry
feature_id,,
node/122546577,{'highway': 'traffic_signals'},POINT (12.45659 41.90025)
node/122548652,"{'crossing': 'zebra', 'highway': 'crossing'}",POINT (12.45213 41.89893)
node/122552836,"{'crossing': 'zebra', 'crossing:island': 'no',...",POINT (12.45057 41.90046)
node/130635170,{'railway': 'tram_level_crossing'},POINT (12.45796 41.90852)
node/130695995,"{'button_operated': 'no', 'crossing': 'traffic...",POINT (12.4553 41.90871)
...,...,...
way/1346768109,"{'footway': 'sidewalk', 'highway': 'footway', ...","LINESTRING (12.45877 41.90575, 12.4588 41.90564)"
way/1346768110,"{'footway': 'sidewalk', 'highway': 'footway', ...","LINESTRING (12.45678 41.90614, 12.45674 41.906..."
way/1348965017,"{'barrier': 'hedge', 'previously': 'way:132155...","LINESTRING (12.45177 41.90162, 12.45165 41.901..."


**Filters for Wineries**

In [4]:
from quackosm._osm_tags_filters import GroupedOsmTagsFilter, OsmTagsFilter
import numpy as np

filter = {
  'craft': ['distillery', 'winery'], 
  'landuse': 'vineyard',
  'shop': 'wine', 
}

area = qosm.geocode_to_geometry('Sonoma')
gdf = qosm.convert_geometry_to_geodataframe(
    area, tags_filter=filter,
    keep_all_tags=True,
    explode_tags=True
)
np.unique(gdf[gdf.name.notna()][['name','craft', 'landuse', 'shop']]['name'])

array(['4th Street cellars', 'Anaba Winery', 'Annapolis Winery',
       'BR Cohn Winery', 'Bartholomew Park Winery', 'Bedrock Vineyard',
       'Bella Vineyards', 'Benovia Winery', 'Bezinger Family Winery',
       'Black Kite Cellars', 'Brown Dog Vineyards',
       'Campobello Family Vine', 'Chateau St. Jean Winery', 'Cirq Estate',
       'Cleary Ranch Vineyards', 'Cline Cellars',
       'D & L Carinalli Vineyards', 'DaVero Farms & Winery',
       'Eco Terreno', 'Eric K. James Vineyards Tasting Room',
       'Eric Ross Winery', 'Ferrari-Carano Vineyards and Winery',
       'Flowers', 'Fort Ross Vineyard', 'Gary Farrell', 'Gloria Ferrer ',
       'Guerro Fernandez', 'Gundlach Bundschu Winery',
       'Gustafson Family Vineyards', 'Hanson of Sonoma Distillery',
       'Hartford Family Winery', 'Homewood Winery',
       'Hop Kiln Winery (HKG Estate)', 'Iron Horse Vineyards',
       'J Rochioli Vineyards & Winery', 'J. Rickards Winery',
       'Jacuzzi Family Winery', 'Kastania', 'Kendall 

In [5]:
m = gdf[gdf.name.notna()].explore(color="orangered", tiles="OpenStreetMap", tooltip=['name'])
gpd.GeoSeries([area], crs=4326).boundary.explore(m=m)

In [6]:
gdf = qosm.convert_geometry_to_geodataframe(
    area, tags_filter={'name': 'Silver*'},
    keep_all_tags=True,
    explode_tags=True
)
gdf[['name', 'craft']]

,name,craft
feature_id,,
node/6499125044,Silver Moon,None
node/358813443,Silver Shoon Ranch 2422 Dam,None
way/242540179,Silver Fox Court,None
way/7720982,Silver Drive,None
way/7722175,Silverwood Street,None
way/7722561,Silver Lake Drive,None
way/39083066,Silver Stone Lane,None
way/39276869,Silver Spur Drive,None
way/7707497,Silver Creek Circle,None


**Points of Interest Queries**

In [7]:
area = qosm.geocode_to_geometry("Barcelona")
gdf = qosm.convert_geometry_to_geodataframe(
    area, tags_filter={"amenity": "bicycle_rental"}
)
gdf

,amenity,geometry
feature_id,,
node/2434563721,bicycle_rental,POINT (2.17733 41.38167)
node/2434563722,bicycle_rental,POINT (2.17679 41.37855)
node/2434567105,bicycle_rental,POINT (2.18223 41.38059)
node/2440208126,bicycle_rental,POINT (2.17704 41.37681)
node/2443607899,bicycle_rental,POINT (2.16364 41.3858)
...,...,...
node/4401863491,bicycle_rental,POINT (2.18092 41.39122)
node/4401882390,bicycle_rental,POINT (2.18068 41.39132)
node/4412467922,bicycle_rental,POINT (2.17414 41.40588)


In [8]:
tags_filter={
  'addr:*': True,
  'name': True,
  'name:en': True,
  'name:official_name': True,
  'name:loc_name': True,
  'craft': 'winery'
}

def fetch(country, filter = tags_filter):
  area = qosm.geocode_to_geometry(country)
  gdf = qosm.convert_geometry_to_geodataframe(
      area, tags_filter=filter,
      explode_tags=True,
      keep_all_tags=False,
      geometry_coverage_iou_threshold=1e-6,
      allow_uncovered_geometry=True,
  )
  return area, gdf

**California Wineries**

In [ ]:
wineries = ['Hafner']

area, gdf = fetch('Sonoma')
gdf

In [ ]:
m = gdf.explore(color="orangered", tiles="CartoDB positron")
gpd.GeoSeries([area], crs=4326).boundary.explore(m=m)

**French Wineries**

In [ ]:
area, gdf = fetch('France')
gdf

In [ ]:
m = gdf.explore(color="orangered", tiles="CartoDB positron")
gpd.GeoSeries([area], crs=4326).boundary.explore(m=m)

**Italian Wineries**

In [ ]:
wineries = ['Nicosia', 'Masseria Setteporte', 'Terre di Giurfo']

area, gdf = fetch('Italy')
gdf


In [ ]:
m = gdf.explore(color="orangered", tiles="CartoDB positron")
gpd.GeoSeries([area], crs=4326).boundary.explore(m=m)